In [1]:
from sqlalchemy import create_engine
from config import Config

In [20]:
r = eng.execute('select * from if_heat')

In [3]:
eng = create_engine(Config.DATABASE_URI)

In [21]:
for row in r.fetchall():
    print(row)

'68508069c943b5b7b0ee98c0a08cf68fb4bfc7c21b08c73d04dcb63c793b1374', 10, 'Norbert Krakowiak', '1', 'Bartosz Smektała', '2', 'Patryk Dudek', '0', 'Jaimon Lidsey', '3', datetime.datetime(2021, 6, 6, 16, 27, 16, 414885))
(34071, '68508069c943b5b7b0ee98c0a08cf68fb4bfc7c21b08c73d04dcb63c793b1374', 11, 'Jaimon Lidsey', '1', 'Piotr Protasiewicz', '3', 'Emil Sayfutdinov', '2', 'Rohan Tungate', '0', datetime.datetime(2021, 6, 6, 16, 27, 17, 99152))
(34072, '68508069c943b5b7b0ee98c0a08cf68fb4bfc7c21b08c73d04dcb63c793b1374', 12, 'Martin Vaculik', '3', 'Dominik Kubera', '1', 'Mateusz Tonder', '0', 'Bartosz Smektała', '2', datetime.datetime(2021, 6, 6, 16, 27, 17, 792057))
(34073, '68508069c943b5b7b0ee98c0a08cf68fb4bfc7c21b08c73d04dcb63c793b1374', 13, 'Janusz Kołodziej', '2', 'Patryk Dudek', '0', 'Piotr Pawlicki', '3', 'Michael Jepsen Jensen', '1', datetime.datetime(2021, 6, 6, 16, 27, 18, 377116))
(34074, '68508069c943b5b7b0ee98c0a08cf68fb4bfc7c21b08c73d04dcb63c793b1374', 14, 'Michael Jepsen Jensen